In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Feb 7 17:11:49 2024

@author: acmsavazzi
"""
#%% HARMONIE_plot.py

#%%                             Libraries
###############################################################################
import numpy as np
import pandas as pd
import xarray as xr
import os
# from xhistogram.xarray import histogram
import netCDF4
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm
from matplotlib.ticker import MaxNLocator, FuncFormatter
from datetime import datetime, timedelta
from netCDF4 import Dataset
import matplotlib.pylab as pylab
from skimage import measure
params = {'legend.fontsize': 'large',
         'axes.labelsize': 24,
         'axes.titlesize':'large',
         'xtick.labelsize':20,
         'ytick.labelsize':20,
         'figure.figsize':[10,7],
         'figure.titlesize':24}
pylab.rcParams.update(params)
import dask
import dask.array as da
dask.config.set({"array.slicing.split_large_chunks": True})
from intake import open_catalog

In [2]:
exps     = ['HA43h22tg3_clim_noHGTQS','HA43h22tg3_clim_noHGTQS_noUVmix','HA43h22tg3_clim_noHGTQS_noSHAL'] 
# col=['k','r','g']
col = ['k','#D04848','#6895D2']
# col_obs=['#4483d5','#64bedb']
col_obs=['#F3B95F','#FDE767']

sty=['-','--',':']
lab = ['Control','UV-OFF', 'SC-OFF']

srt_time    = np.datetime64('2020-01-03T00:30')
end_time    = np.datetime64('2020-02-29T23')
# define (sub)cloud layer in km
sc_layer_base = 0 
sc_layer_top = 0.6
c_layer_base = 0.9
c_layer_top = 1.5

my_data_dir = os.path.abspath('../data/')+'/'
figure_dir  = os.path.abspath('../figures/')+'/'

In [3]:
# Import Observations
cat = open_catalog("https://raw.githubusercontent.com/eurec4a/eurec4a-intake/master/catalog.yml")
#
joanne = cat.dropsondes.JOANNE.level3.to_dask()
joanne['alt'] = joanne['alt']/1000 # Height from m to km 
joanne = joanne.rename(name_dict={'theta':'thv'})
#
radio_rb = cat.radiosondes.ronbrown.to_dask()
radio_rb = radio_rb.rename(name_dict={'theta':'thv'})
radio_mer = cat.radiosondes.ms_merian.to_dask()
radio_mer = radio_mer.rename(name_dict={'theta':'thv'})
radio_met = cat.radiosondes.meteor.to_dask()
radio_met = radio_met.rename(name_dict={'theta':'thv'})
radio_bco = cat.radiosondes.bco.to_dask()
radio_bco = radio_bco.rename(name_dict={'theta':'thv'})
#
# meteor_flx = cat.Meteor.surface_fluxes.to_dask()

/Users/acmsavazzi/opt/anaconda3/envs/phdproject2/lib/python3.11/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(
/Users/acmsavazzi/opt/anaconda3/envs/phdproject2/lib/python3.11/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(
/Users/acmsavazzi/opt/anaconda3/envs/phdproject2/lib/python3.11/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(
/Users/acmsavazzi/opt/anaconda3/envs/phdproject2/lib/python3.11/site-packages/xarray/coding/times.py:254: RuntimeWarning: invalid value encountered in cast
  flat_num_dates_ns_int = (flat_num_dates * _NS_PER_TIME_DELTA[delta]).astype(
/Users/acmsavazzi/opt/anaconda3/envs/phdproject2/lib/python3

In [4]:
# Import experiments
harm2d = {}
harm2d_synopt = {}
harm_obj_synopt = {}
harm_obj = {}
ds_org_4km_synopt={}
harm_srf_sml = {}
harm_srf_sml_synopt = {}
harm3d_snap={}
ds_org_smoc_mean = {}
ds_org_smoc_mean_synopt = {}
harm_subsamp = {}
for exp in exps:
    harm2d[exp] = xr.open_mfdataset(my_data_dir+exp+'/'+exp[16:]+'_harm2d_200.nc',\
                                          combine='by_coords',chunks={'time':24*10})
    harm2d[exp]['z'] = harm2d[exp]['z']/1000  ## height in km 

    harm2d_synopt[exp] = xr.open_mfdataset(my_data_dir+exp+'/'+exp[16:]+'_harm2d_synopt.nc',\
                                          combine='by_coords',chunks={'time':24*10})
    harm2d_synopt[exp]['z'] = harm2d_synopt[exp]['z']/1000  ## height in km 

    harm_obj_synopt[exp] = xr.open_mfdataset(my_data_dir+exp+'/'+exp[16:]+'_harm_obj_synopt.nc',\
                                          combine='by_coords',chunks={'time':24*10})
    #
    ds_org_4km_synopt[exp] = xr.open_mfdataset(my_data_dir+exp+'/'+exp[16:]+'_ds_org_4km_synopt.nc',\
                                          combine='by_coords',chunks={'time':24*10})

    harm_obj[exp] = xr.open_mfdataset(my_data_dir+exp+'/'+exp[16:]+'_obj_stat.nc',\
                                          combine='by_coords',chunks={'time':24*10})

    harm_srf_sml[exp] = xr.open_mfdataset(my_data_dir+exp+'/'+exp[16:]+'_harm_srf_sml.nc',\
                                          combine='by_coords',chunks={'time':24*10})
    #
    harm_srf_sml_synopt[exp] = harm_srf_sml[exp].mean(dim=('x','y')).chunk(dict(time=-1)).\
                        interpolate_na(dim='time').rolling(time=32,center=True).mean()

    # harm 3d snapshot
    harm3d_snap[exp] = xr.open_mfdataset(my_data_dir+exp+'/'+exp[16:]+'_harm3d_snapshots.nc',\
                                      combine='by_coords')


    ds_org_smoc_mean[exp] = xr.open_mfdataset(my_data_dir+exp+'/'+exp[16:]+'_smoc_metrics.nc',\
                                          combine='by_coords',chunks={'time':-1})
    ds_org_smoc_mean_synopt[exp] = ds_org_smoc_mean[exp].chunk(dict(time=-1)).interpolate_na(dim='time').rolling(time=32,center=True).mean()

    # subsampled 3d fields
    harm_subsamp[exp] = xr.open_mfdataset(my_data_dir+exp+'/'+exp[16:]+'_harm_subsamp.nc',\
                                      combine='by_coords')
    harm_subsamp[exp]['z']=harm_subsamp[exp]['z']/1000

In [5]:
time_CC_exp_is_larger= {}
time_CC_exp_is_smaller= {}
for exp in exps[1:]:
    time_CC_exp_is_larger[exp] = ds_org_4km_synopt[exp].where(((ds_org_4km_synopt[exp] - \
                                                         ds_org_4km_synopt['HA43h22tg3_clim_noHGTQS'])['cloud_fraction']>0).compute(),drop=True).time
    time_CC_exp_is_smaller[exp] = ds_org_4km_synopt[exp].where(((ds_org_4km_synopt[exp] - \
                                                         ds_org_4km_synopt['HA43h22tg3_clim_noHGTQS'])['cloud_fraction']<=0).compute(),drop=True).time